In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,EarlyStopping

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 - band_2)/2
        if row['inc_angle'] == 'na':
            ang = -1
        else:
            ang = float(row['inc_angle'])
        band_3 = band_3*ang

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(16, (3, 3), padding='same',input_shape=(75,75,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(16, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model
print('model model')


model model


In [5]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch<10:
        return 0.0008
    elif epoch<30:
        return 0.0004
    else:
        return 0.0001
    

def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=120, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)

Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.57438, saving model to best_m.h5
 - 2s - loss: 0.7023 - acc: 0.5905 - val_loss: 0.5744 - val_acc: 0.7028
Epoch 2/120
Epoch 00002: val_loss improved from 0.57438 to 0.52876, saving model to best_m.h5
 - 1s - loss: 0.5612 - acc: 0.6928 - val_loss: 0.5288 - val_acc: 0.7402
Epoch 3/120
Epoch 00003: val_loss improved from 0.52876 to 0.49698, saving model to best_m.h5
 - 2s - loss: 0.4986 - acc: 0.7379 - val_loss: 0.4970 - val_acc: 0.7607
Epoch 4/120
Epoch 00004: val_loss improved from 0.49698 to 0.47431, saving model to best_m.h5
 - 1s - loss: 0.4790 - acc: 0.7534 - val_loss: 0.4743 - val_acc: 0.8168
Epoch 5/120
Epoch 00005: val_loss improved from 0.47431 to 0.40304, saving model to best_m.h5
 - 1s - loss: 0.4360 - acc: 0.8031 - val_loss: 0.4030 - val_acc: 0.8411
Epoch 6/120
Epoch 00006: val_loss improved from 0.40304 to 0.39206, saving model to best_m.h5
 - 1s - loss: 0.4254 - acc: 0.8087 - val_loss: 0.3921 - val_acc: 0.8430
Epoch 7/

Epoch 60/120
Epoch 00060: val_loss improved from 0.20312 to 0.20201, saving model to best_m.h5
 - 1s - loss: 0.2058 - acc: 0.9098 - val_loss: 0.2020 - val_acc: 0.9196
Epoch 61/120
Epoch 00061: val_loss did not improve
 - 1s - loss: 0.1944 - acc: 0.9167 - val_loss: 0.2045 - val_acc: 0.9196
Epoch 62/120
Epoch 00062: val_loss improved from 0.20201 to 0.20018, saving model to best_m.h5
 - 1s - loss: 0.2081 - acc: 0.9138 - val_loss: 0.2002 - val_acc: 0.9159
Epoch 63/120
Epoch 00063: val_loss did not improve
 - 1s - loss: 0.1932 - acc: 0.9160 - val_loss: 0.2026 - val_acc: 0.9327
Epoch 64/120
Epoch 00064: val_loss did not improve
 - 1s - loss: 0.2188 - acc: 0.9044 - val_loss: 0.2021 - val_acc: 0.9234
Epoch 65/120
Epoch 00065: val_loss did not improve
 - 1s - loss: 0.1950 - acc: 0.9151 - val_loss: 0.2186 - val_acc: 0.9103
Epoch 66/120
Epoch 00066: val_loss did not improve
 - 1s - loss: 0.1895 - acc: 0.9176 - val_loss: 0.2105 - val_acc: 0.9215
Epoch 67/120
Epoch 00067: val_loss did not improve


Epoch 00003: val_loss improved from 0.68375 to 0.59804, saving model to best_m.h5
 - 1s - loss: 0.6052 - acc: 0.6613 - val_loss: 0.5980 - val_acc: 0.6879
Epoch 4/120
Epoch 00004: val_loss improved from 0.59804 to 0.43144, saving model to best_m.h5
 - 1s - loss: 0.5221 - acc: 0.7283 - val_loss: 0.4314 - val_acc: 0.8019
Epoch 5/120
Epoch 00005: val_loss did not improve
 - 1s - loss: 0.4903 - acc: 0.7543 - val_loss: 0.4593 - val_acc: 0.7458
Epoch 6/120
Epoch 00006: val_loss improved from 0.43144 to 0.34016, saving model to best_m.h5
 - 1s - loss: 0.4502 - acc: 0.7982 - val_loss: 0.3402 - val_acc: 0.8505
Epoch 7/120
Epoch 00007: val_loss did not improve
 - 1s - loss: 0.4300 - acc: 0.8043 - val_loss: 0.3483 - val_acc: 0.8336
Epoch 8/120
Epoch 00008: val_loss improved from 0.34016 to 0.31666, saving model to best_m.h5
 - 1s - loss: 0.4226 - acc: 0.8062 - val_loss: 0.3167 - val_acc: 0.8636
Epoch 9/120
Epoch 00009: val_loss improved from 0.31666 to 0.31397, saving model to best_m.h5
 - 1s - lo

Epoch 62/120
Epoch 00062: val_loss did not improve
 - 1s - loss: 0.1972 - acc: 0.9183 - val_loss: 0.1873 - val_acc: 0.9252
Epoch 63/120
Epoch 00063: val_loss did not improve
 - 1s - loss: 0.2292 - acc: 0.8985 - val_loss: 0.2016 - val_acc: 0.9215
Epoch 64/120
Epoch 00064: val_loss did not improve
 - 1s - loss: 0.2255 - acc: 0.9077 - val_loss: 0.1861 - val_acc: 0.9252
Epoch 65/120
Epoch 00065: val_loss improved from 0.17900 to 0.17826, saving model to best_m.h5
 - 1s - loss: 0.2131 - acc: 0.9001 - val_loss: 0.1783 - val_acc: 0.9252
Epoch 66/120
Epoch 00066: val_loss did not improve
 - 1s - loss: 0.1999 - acc: 0.9091 - val_loss: 0.1867 - val_acc: 0.9252
Epoch 67/120
Epoch 00067: val_loss improved from 0.17826 to 0.17707, saving model to best_m.h5
 - 1s - loss: 0.2258 - acc: 0.9011 - val_loss: 0.1771 - val_acc: 0.9327
Epoch 68/120
Epoch 00068: val_loss improved from 0.17707 to 0.17627, saving model to best_m.h5
 - 1s - loss: 0.1990 - acc: 0.9183 - val_loss: 0.1763 - val_acc: 0.9290
Epoch 6

Epoch 4/120
Epoch 00004: val_loss improved from 0.46472 to 0.39987, saving model to best_m.h5
 - 1s - loss: 0.4719 - acc: 0.7665 - val_loss: 0.3999 - val_acc: 0.8127
Epoch 5/120
Epoch 00005: val_loss improved from 0.39987 to 0.29691, saving model to best_m.h5
 - 1s - loss: 0.4091 - acc: 0.8102 - val_loss: 0.2969 - val_acc: 0.8820
Epoch 6/120
Epoch 00006: val_loss did not improve
 - 1s - loss: 0.3932 - acc: 0.8229 - val_loss: 0.3644 - val_acc: 0.8689
Epoch 7/120
Epoch 00007: val_loss improved from 0.29691 to 0.28810, saving model to best_m.h5
 - 1s - loss: 0.3555 - acc: 0.8463 - val_loss: 0.2881 - val_acc: 0.8727
Epoch 8/120
Epoch 00008: val_loss did not improve
 - 1s - loss: 0.3545 - acc: 0.8452 - val_loss: 0.2960 - val_acc: 0.8820
Epoch 9/120
Epoch 00009: val_loss improved from 0.28810 to 0.26522, saving model to best_m.h5
 - 1s - loss: 0.3096 - acc: 0.8642 - val_loss: 0.2652 - val_acc: 0.8727
Epoch 10/120
Epoch 00010: val_loss improved from 0.26522 to 0.26051, saving model to best_m.

Epoch 66/120
Epoch 00066: val_loss did not improve
 - 1s - loss: 0.1736 - acc: 0.9249 - val_loss: 0.2228 - val_acc: 0.9082
Epoch 67/120
Epoch 00067: val_loss did not improve
 - 1s - loss: 0.1942 - acc: 0.9223 - val_loss: 0.2444 - val_acc: 0.9045
Epoch 68/120
Epoch 00068: val_loss did not improve
 - 1s - loss: 0.1962 - acc: 0.9148 - val_loss: 0.2167 - val_acc: 0.9120
Epoch 69/120
Epoch 00069: val_loss did not improve
 - 1s - loss: 0.1871 - acc: 0.9229 - val_loss: 0.2181 - val_acc: 0.9007
Epoch 70/120
Epoch 00070: val_loss did not improve
 - 1s - loss: 0.1739 - acc: 0.9240 - val_loss: 0.2173 - val_acc: 0.9082
Epoch 71/120
Epoch 00071: val_loss did not improve
 - 1s - loss: 0.1751 - acc: 0.9190 - val_loss: 0.2255 - val_acc: 0.9082
Epoch 72/120
Epoch 00072: val_loss did not improve
 - 1s - loss: 0.1758 - acc: 0.9205 - val_loss: 0.2119 - val_acc: 0.9064
Epoch 73/120
Epoch 00073: val_loss did not improve
 - 1s - loss: 0.1921 - acc: 0.9205 - val_loss: 0.2131 - val_acc: 0.9064
Epoch 74/120
Epo

In [6]:
import pickle
with open('../features/cnn_1_aug_ang_fold3_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))


submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_1_aug_ang_fold3.csv', index=False)

0.18610606276
         id  is_iceberg
0  5941774d    0.115259
1  4023181e    0.994135
2  b20200e4    0.994109
3  e7f018bb    0.994135
4  4371c8c3    0.976975


In [7]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(32, (3, 3),input_shape=(75, 75, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))
    return model
print('model model')

model model


In [9]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=120, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)

Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.56800, saving model to best_m.h5
 - 2s - loss: 0.6293 - acc: 0.6184 - val_loss: 0.5680 - val_acc: 0.7308
Epoch 2/120
Epoch 00002: val_loss did not improve
 - 1s - loss: 0.5407 - acc: 0.7039 - val_loss: 0.5925 - val_acc: 0.7159
Epoch 3/120
Epoch 00003: val_loss improved from 0.56800 to 0.37327, saving model to best_m.h5
 - 1s - loss: 0.4709 - acc: 0.7666 - val_loss: 0.3733 - val_acc: 0.8411
Epoch 4/120
Epoch 00004: val_loss did not improve
 - 1s - loss: 0.3910 - acc: 0.8357 - val_loss: 0.3943 - val_acc: 0.8206
Epoch 5/120
Epoch 00005: val_loss improved from 0.37327 to 0.33092, saving model to best_m.h5
 - 1s - loss: 0.3828 - acc: 0.8355 - val_loss: 0.3309 - val_acc: 0.8561
Epoch 6/120
Epoch 00006: val_loss did not improve
 - 2s - loss: 0.3639 - acc: 0.8338 - val_loss: 0.4556 - val_acc: 0.7907
Epoch 7/120
Epoch 00007: val_loss did not improve
 - 1s - loss: 0.3381 - acc: 0.8604 - val_loss: 0.3365 - val_acc: 0.8579
Epoch 8/120
Epoch 

Epoch 62/120
Epoch 00062: val_loss did not improve
 - 1s - loss: 0.1578 - acc: 0.9375 - val_loss: 0.1944 - val_acc: 0.9121
Epoch 63/120
Epoch 00063: val_loss did not improve
 - 1s - loss: 0.1567 - acc: 0.9363 - val_loss: 0.1944 - val_acc: 0.9121
Epoch 64/120
Epoch 00064: val_loss did not improve
 - 1s - loss: 0.1517 - acc: 0.9356 - val_loss: 0.1923 - val_acc: 0.9140
Epoch 65/120
Epoch 00065: val_loss did not improve
 - 1s - loss: 0.1606 - acc: 0.9337 - val_loss: 0.2077 - val_acc: 0.8991
Epoch 66/120
Epoch 00066: val_loss did not improve
 - 1s - loss: 0.1627 - acc: 0.9394 - val_loss: 0.1970 - val_acc: 0.9084
Epoch 67/120
Epoch 00067: val_loss did not improve
 - 1s - loss: 0.1515 - acc: 0.9406 - val_loss: 0.1953 - val_acc: 0.9178
Epoch 68/120
Epoch 00068: val_loss did not improve
 - 1s - loss: 0.1701 - acc: 0.9354 - val_loss: 0.2072 - val_acc: 0.8972
Epoch 69/120
Epoch 00069: val_loss did not improve
 - 1s - loss: 0.1634 - acc: 0.9373 - val_loss: 0.1946 - val_acc: 0.9103
Epoch 70/120
Epo

Epoch 7/120
Epoch 00007: val_loss improved from 0.31094 to 0.30454, saving model to best_m.h5
 - 1s - loss: 0.3603 - acc: 0.8424 - val_loss: 0.3045 - val_acc: 0.8748
Epoch 8/120
Epoch 00008: val_loss improved from 0.30454 to 0.29050, saving model to best_m.h5
 - 1s - loss: 0.3052 - acc: 0.8660 - val_loss: 0.2905 - val_acc: 0.8654
Epoch 9/120
Epoch 00009: val_loss improved from 0.29050 to 0.25043, saving model to best_m.h5
 - 1s - loss: 0.3343 - acc: 0.8551 - val_loss: 0.2504 - val_acc: 0.8916
Epoch 10/120
Epoch 00010: val_loss did not improve
 - 1s - loss: 0.3183 - acc: 0.8606 - val_loss: 0.2769 - val_acc: 0.8879
Epoch 11/120
Epoch 00011: val_loss improved from 0.25043 to 0.20901, saving model to best_m.h5
 - 1s - loss: 0.2872 - acc: 0.8796 - val_loss: 0.2090 - val_acc: 0.9047
Epoch 12/120
Epoch 00012: val_loss did not improve
 - 1s - loss: 0.2677 - acc: 0.8738 - val_loss: 0.2169 - val_acc: 0.9084
Epoch 13/120
Epoch 00013: val_loss did not improve
 - 1s - loss: 0.2457 - acc: 0.8895 - v

Epoch 68/120
Epoch 00068: val_loss did not improve
 - 1s - loss: 0.1879 - acc: 0.9160 - val_loss: 0.1987 - val_acc: 0.9084
Epoch 69/120
Epoch 00069: val_loss did not improve
 - 1s - loss: 0.1682 - acc: 0.9229 - val_loss: 0.1878 - val_acc: 0.9234
Epoch 70/120
Epoch 00070: val_loss did not improve
 - 1s - loss: 0.1712 - acc: 0.9325 - val_loss: 0.1784 - val_acc: 0.9196
Epoch 71/120
Epoch 00071: val_loss did not improve
 - 1s - loss: 0.1748 - acc: 0.9226 - val_loss: 0.1820 - val_acc: 0.9196
Epoch 72/120
Epoch 00072: val_loss did not improve
 - 1s - loss: 0.1711 - acc: 0.9290 - val_loss: 0.1812 - val_acc: 0.9215
Epoch 73/120
Epoch 00073: val_loss did not improve
 - 1s - loss: 0.1736 - acc: 0.9318 - val_loss: 0.1743 - val_acc: 0.9234
Epoch 74/120
Epoch 00074: val_loss did not improve
 - 1s - loss: 0.1712 - acc: 0.9333 - val_loss: 0.1778 - val_acc: 0.9159
Epoch 75/120
Epoch 00075: val_loss did not improve
 - 1s - loss: 0.1506 - acc: 0.9470 - val_loss: 0.1747 - val_acc: 0.9234
Epoch 76/120
Epo

Epoch 00011: val_loss did not improve
 - 1s - loss: 0.2699 - acc: 0.8708 - val_loss: 0.2962 - val_acc: 0.8745
Epoch 12/120
Epoch 00012: val_loss improved from 0.28611 to 0.27304, saving model to best_m.h5
 - 1s - loss: 0.2261 - acc: 0.9002 - val_loss: 0.2730 - val_acc: 0.8820
Epoch 13/120
Epoch 00013: val_loss improved from 0.27304 to 0.26225, saving model to best_m.h5
 - 1s - loss: 0.2199 - acc: 0.8995 - val_loss: 0.2623 - val_acc: 0.8970
Epoch 14/120
Epoch 00014: val_loss did not improve
 - 1s - loss: 0.2003 - acc: 0.9110 - val_loss: 0.2739 - val_acc: 0.9045
Epoch 15/120
Epoch 00015: val_loss did not improve
 - 1s - loss: 0.2412 - acc: 0.8877 - val_loss: 0.2628 - val_acc: 0.9007
Epoch 16/120
Epoch 00016: val_loss improved from 0.26225 to 0.25059, saving model to best_m.h5
 - 1s - loss: 0.2258 - acc: 0.8923 - val_loss: 0.2506 - val_acc: 0.9026
Epoch 17/120
Epoch 00017: val_loss did not improve
 - 1s - loss: 0.2091 - acc: 0.9110 - val_loss: 0.2679 - val_acc: 0.8970
Epoch 18/120
Epoch 0

Epoch 75/120
Epoch 00075: val_loss did not improve
 - 1s - loss: 0.1440 - acc: 0.9347 - val_loss: 0.2440 - val_acc: 0.9232
Epoch 76/120
Epoch 00076: val_loss did not improve
 - 1s - loss: 0.1562 - acc: 0.9299 - val_loss: 0.2305 - val_acc: 0.9288
Epoch 77/120
Epoch 00077: val_loss did not improve
 - 1s - loss: 0.1453 - acc: 0.9353 - val_loss: 0.2365 - val_acc: 0.9251
Epoch 78/120
Epoch 00078: val_loss did not improve
 - 1s - loss: 0.1420 - acc: 0.9375 - val_loss: 0.2338 - val_acc: 0.9307
Epoch 79/120
Epoch 00079: val_loss did not improve
 - 1s - loss: 0.1376 - acc: 0.9355 - val_loss: 0.2494 - val_acc: 0.9270
Epoch 80/120
Epoch 00080: val_loss did not improve
 - 1s - loss: 0.1354 - acc: 0.9391 - val_loss: 0.2481 - val_acc: 0.9307
Epoch 81/120
Epoch 00081: val_loss improved from 0.22971 to 0.22161, saving model to best_m.h5
 - 1s - loss: 0.1663 - acc: 0.9249 - val_loss: 0.2216 - val_acc: 0.9213
Epoch 82/120
Epoch 00082: val_loss did not improve
 - 1s - loss: 0.1288 - acc: 0.9497 - val_los

In [10]:
with open('../features/cnn_2_aug_ang_fold3_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))

# this 2728
# skimage 2710

submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_2_aug_ang_fold3.csv', index=False)

0.186746431198
         id  is_iceberg
0  5941774d    0.095711
1  4023181e    0.815958
2  b20200e4    0.170421
3  e7f018bb    0.995130
4  4371c8c3    0.062769


In [11]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))
    return model
print('model model')

model model


In [12]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=120, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)

Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.62714, saving model to best_m.h5
 - 3s - loss: 0.6758 - acc: 0.5627 - val_loss: 0.6271 - val_acc: 0.6467
Epoch 2/120
Epoch 00002: val_loss improved from 0.62714 to 0.55740, saving model to best_m.h5
 - 2s - loss: 0.5695 - acc: 0.6989 - val_loss: 0.5574 - val_acc: 0.7589
Epoch 3/120
Epoch 00003: val_loss did not improve
 - 2s - loss: 0.5185 - acc: 0.7177 - val_loss: 0.6092 - val_acc: 0.6953
Epoch 4/120
Epoch 00004: val_loss improved from 0.55740 to 0.42343, saving model to best_m.h5
 - 2s - loss: 0.5003 - acc: 0.7474 - val_loss: 0.4234 - val_acc: 0.8355
Epoch 5/120
Epoch 00005: val_loss improved from 0.42343 to 0.37141, saving model to best_m.h5
 - 2s - loss: 0.4431 - acc: 0.7989 - val_loss: 0.3714 - val_acc: 0.8523
Epoch 6/120
Epoch 00006: val_loss did not improve
 - 2s - loss: 0.4088 - acc: 0.8049 - val_loss: 0.3813 - val_acc: 0.8299
Epoch 7/120
Epoch 00007: val_loss improved from 0.37141 to 0.34503, saving model to best_m.h5
 -

Epoch 61/120
Epoch 00061: val_loss improved from 0.18925 to 0.18334, saving model to best_m.h5
 - 2s - loss: 0.2015 - acc: 0.9148 - val_loss: 0.1833 - val_acc: 0.9271
Epoch 62/120
Epoch 00062: val_loss did not improve
 - 2s - loss: 0.1920 - acc: 0.9186 - val_loss: 0.1984 - val_acc: 0.9159
Epoch 63/120
Epoch 00063: val_loss did not improve
 - 2s - loss: 0.2038 - acc: 0.9195 - val_loss: 0.1980 - val_acc: 0.9178
Epoch 64/120
Epoch 00064: val_loss did not improve
 - 2s - loss: 0.1984 - acc: 0.9079 - val_loss: 0.2232 - val_acc: 0.9103
Epoch 65/120
Epoch 00065: val_loss did not improve
 - 2s - loss: 0.1927 - acc: 0.9136 - val_loss: 0.1968 - val_acc: 0.9196
Epoch 66/120
Epoch 00066: val_loss did not improve
 - 2s - loss: 0.1918 - acc: 0.9233 - val_loss: 0.2490 - val_acc: 0.8991
Epoch 67/120
Epoch 00067: val_loss did not improve
 - 2s - loss: 0.1852 - acc: 0.9240 - val_loss: 0.2031 - val_acc: 0.9140
Epoch 68/120
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.2056 - acc: 0.9134 - val_los

Epoch 6/120
Epoch 00006: val_loss improved from 0.39960 to 0.34928, saving model to best_m.h5
 - 2s - loss: 0.4198 - acc: 0.8154 - val_loss: 0.3493 - val_acc: 0.8355
Epoch 7/120
Epoch 00007: val_loss did not improve
 - 2s - loss: 0.3718 - acc: 0.8229 - val_loss: 0.3769 - val_acc: 0.8243
Epoch 8/120
Epoch 00008: val_loss improved from 0.34928 to 0.33383, saving model to best_m.h5
 - 2s - loss: 0.3647 - acc: 0.8330 - val_loss: 0.3338 - val_acc: 0.8430
Epoch 9/120
Epoch 00009: val_loss improved from 0.33383 to 0.26269, saving model to best_m.h5
 - 2s - loss: 0.3926 - acc: 0.8246 - val_loss: 0.2627 - val_acc: 0.8785
Epoch 10/120
Epoch 00010: val_loss did not improve
 - 2s - loss: 0.3511 - acc: 0.8511 - val_loss: 0.2810 - val_acc: 0.8598
Epoch 11/120
Epoch 00011: val_loss did not improve
 - 2s - loss: 0.3140 - acc: 0.8589 - val_loss: 0.3193 - val_acc: 0.8505
Epoch 12/120
Epoch 00012: val_loss improved from 0.26269 to 0.24156, saving model to best_m.h5
 - 2s - loss: 0.2825 - acc: 0.8800 - va

Epoch 67/120
Epoch 00067: val_loss did not improve
 - 2s - loss: 0.2087 - acc: 0.9151 - val_loss: 0.1908 - val_acc: 0.9290
Epoch 68/120
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.1937 - acc: 0.9208 - val_loss: 0.2025 - val_acc: 0.9140
Epoch 69/120
Epoch 00069: val_loss did not improve
 - 2s - loss: 0.1971 - acc: 0.9193 - val_loss: 0.1964 - val_acc: 0.9234
Epoch 70/120
Epoch 00070: val_loss did not improve
 - 2s - loss: 0.1831 - acc: 0.9212 - val_loss: 0.2029 - val_acc: 0.9084
Epoch 71/120
Epoch 00071: val_loss did not improve
 - 2s - loss: 0.2124 - acc: 0.9266 - val_loss: 0.1812 - val_acc: 0.9271
Epoch 72/120
Epoch 00072: val_loss did not improve
 - 2s - loss: 0.1906 - acc: 0.9176 - val_loss: 0.1931 - val_acc: 0.9196
Epoch 73/120
Epoch 00073: val_loss did not improve
 - 2s - loss: 0.1961 - acc: 0.9205 - val_loss: 0.1905 - val_acc: 0.9271
Epoch 74/120
Epoch 00074: val_loss did not improve
 - 2s - loss: 0.1896 - acc: 0.9233 - val_loss: 0.1787 - val_acc: 0.9308
Epoch 75/120
Epo

Epoch 11/120
Epoch 00011: val_loss improved from 0.32215 to 0.26250, saving model to best_m.h5
 - 2s - loss: 0.3130 - acc: 0.8588 - val_loss: 0.2625 - val_acc: 0.8989
Epoch 12/120
Epoch 00012: val_loss improved from 0.26250 to 0.26149, saving model to best_m.h5
 - 2s - loss: 0.2819 - acc: 0.8690 - val_loss: 0.2615 - val_acc: 0.8933
Epoch 13/120
Epoch 00013: val_loss did not improve
 - 2s - loss: 0.2593 - acc: 0.8854 - val_loss: 0.2915 - val_acc: 0.8951
Epoch 14/120
Epoch 00014: val_loss improved from 0.26149 to 0.25687, saving model to best_m.h5
 - 2s - loss: 0.2619 - acc: 0.8964 - val_loss: 0.2569 - val_acc: 0.8989
Epoch 15/120
Epoch 00015: val_loss improved from 0.25687 to 0.24763, saving model to best_m.h5
 - 2s - loss: 0.2549 - acc: 0.8938 - val_loss: 0.2476 - val_acc: 0.9120
Epoch 16/120
Epoch 00016: val_loss did not improve
 - 2s - loss: 0.2659 - acc: 0.8824 - val_loss: 0.2697 - val_acc: 0.8895
Epoch 17/120
Epoch 00017: val_loss did not improve
 - 2s - loss: 0.2632 - acc: 0.8800 

Epoch 00074: val_loss did not improve
 - 2s - loss: 0.1740 - acc: 0.9366 - val_loss: 0.2455 - val_acc: 0.9120
Epoch 75/120
Epoch 00075: val_loss did not improve
 - 2s - loss: 0.1743 - acc: 0.9280 - val_loss: 0.2258 - val_acc: 0.9232
Epoch 76/120
Epoch 00076: val_loss did not improve
 - 2s - loss: 0.1663 - acc: 0.9356 - val_loss: 0.2573 - val_acc: 0.9026
Epoch 77/120
Epoch 00077: val_loss did not improve
 - 2s - loss: 0.1748 - acc: 0.9186 - val_loss: 0.2378 - val_acc: 0.9251
Epoch 78/120
Epoch 00078: val_loss did not improve
 - 2s - loss: 0.1721 - acc: 0.9191 - val_loss: 0.2311 - val_acc: 0.9288
Epoch 79/120
Epoch 00079: val_loss did not improve
 - 2s - loss: 0.1811 - acc: 0.9232 - val_loss: 0.2307 - val_acc: 0.9307
Epoch 80/120
Epoch 00080: val_loss did not improve
 - 2s - loss: 0.1708 - acc: 0.9203 - val_loss: 0.2443 - val_acc: 0.9176
Epoch 81/120
Epoch 00081: val_loss did not improve
 - 2s - loss: 0.1592 - acc: 0.9366 - val_loss: 0.2765 - val_acc: 0.8989
Epoch 82/120
Epoch 00082: val

In [13]:
with open('../features/cnn_3_aug_ang_fold3_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))


submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_3_aug_ang_fold3.csv', index=False)

0.192085260422
         id  is_iceberg
0  5941774d    0.110032
1  4023181e    0.827832
2  b20200e4    0.323091
3  e7f018bb    0.996253
4  4371c8c3    0.015420


In [ ]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
def ConvBlock(model, layers, filters):
    '''Create [layers] layers consisting of zero padding, a convolution with [filters] 3x3 filters and batch normalization. Perform max pooling after the last layer.'''
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Conv2D(filters, (3, 3), activation='relu'))
        model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()

    # Input image: 75x75x3
    model.add(Lambda(lambda x: x, input_shape=(75, 75, 3)))
    ConvBlock(model, 1, 32)
    # 37x37x32
    ConvBlock(model, 1, 64)
    # 18x18x64
    ConvBlock(model, 1, 128)
    # 9x9x128
    ConvBlock(model, 1, 128)
    # 4x4x128
    model.add(Flatten())
    model.add(Dense(1,activation='sigmoid'))
    
    return model
print('model model')


model model


In [ ]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=120, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)

Epoch 1/120
Epoch 00001: val_loss improved from inf to 0.51158, saving model to best_m.h5
 - 4s - loss: 0.6005 - acc: 0.7385 - val_loss: 0.5116 - val_acc: 0.7794
Epoch 2/120
Epoch 00002: val_loss did not improve
 - 2s - loss: 0.4308 - acc: 0.8151 - val_loss: 0.7116 - val_acc: 0.6336
Epoch 3/120
Epoch 00003: val_loss did not improve
 - 2s - loss: 0.4013 - acc: 0.8253 - val_loss: 1.0048 - val_acc: 0.6336
Epoch 4/120
Epoch 00004: val_loss did not improve
 - 2s - loss: 0.3292 - acc: 0.8507 - val_loss: 0.6256 - val_acc: 0.7626
Epoch 5/120
Epoch 00005: val_loss did not improve
 - 2s - loss: 0.3509 - acc: 0.8564 - val_loss: 0.5121 - val_acc: 0.7963
Epoch 6/120
Epoch 00006: val_loss did not improve
 - 2s - loss: 0.3154 - acc: 0.8539 - val_loss: 0.5246 - val_acc: 0.7981
Epoch 7/120
Epoch 00007: val_loss improved from 0.51158 to 0.25688, saving model to best_m.h5
 - 2s - loss: 0.3084 - acc: 0.8594 - val_loss: 0.2569 - val_acc: 0.8841
Epoch 8/120
Epoch 00008: val_loss did not improve
 - 2s - loss

Epoch 66/120
Epoch 00066: val_loss did not improve
 - 2s - loss: 0.1251 - acc: 0.9564 - val_loss: 0.2103 - val_acc: 0.9196
Epoch 67/120
Epoch 00067: val_loss did not improve
 - 2s - loss: 0.1077 - acc: 0.9591 - val_loss: 0.2642 - val_acc: 0.8972
Epoch 68/120
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.1273 - acc: 0.9536 - val_loss: 0.2426 - val_acc: 0.9159
Epoch 69/120
Epoch 00069: val_loss did not improve
 - 2s - loss: 0.1349 - acc: 0.9449 - val_loss: 0.2686 - val_acc: 0.8991
Epoch 70/120
Epoch 00070: val_loss did not improve
 - 2s - loss: 0.1020 - acc: 0.9621 - val_loss: 0.2393 - val_acc: 0.8991
Epoch 71/120
Epoch 00071: val_loss did not improve
 - 2s - loss: 0.1412 - acc: 0.9477 - val_loss: 0.2456 - val_acc: 0.8991
Epoch 72/120
Epoch 00072: val_loss did not improve
 - 2s - loss: 0.1152 - acc: 0.9526 - val_loss: 0.2628 - val_acc: 0.8972
Epoch 73/120
Epoch 00073: val_loss did not improve
 - 2s - loss: 0.1144 - acc: 0.9572 - val_loss: 0.2482 - val_acc: 0.9047
Epoch 74/120
Epo

Epoch 11/120
Epoch 00011: val_loss did not improve
 - 2s - loss: 0.2170 - acc: 0.9072 - val_loss: 0.2278 - val_acc: 0.8991
Epoch 12/120
Epoch 00012: val_loss did not improve
 - 2s - loss: 0.2415 - acc: 0.9013 - val_loss: 0.2125 - val_acc: 0.9103
Epoch 13/120
Epoch 00013: val_loss did not improve
 - 2s - loss: 0.2611 - acc: 0.8850 - val_loss: 0.2164 - val_acc: 0.8916
Epoch 14/120
Epoch 00014: val_loss did not improve
 - 2s - loss: 0.2046 - acc: 0.9143 - val_loss: 0.2128 - val_acc: 0.9178
Epoch 15/120
Epoch 00015: val_loss improved from 0.20817 to 0.20506, saving model to best_m.h5
 - 2s - loss: 0.2231 - acc: 0.9006 - val_loss: 0.2051 - val_acc: 0.9159
Epoch 16/120
Epoch 00016: val_loss improved from 0.20506 to 0.18245, saving model to best_m.h5
 - 2s - loss: 0.2167 - acc: 0.9011 - val_loss: 0.1825 - val_acc: 0.9178
Epoch 17/120
Epoch 00017: val_loss did not improve
 - 2s - loss: 0.2433 - acc: 0.8994 - val_loss: 0.1938 - val_acc: 0.9121
Epoch 18/120
Epoch 00018: val_loss did not improve


In [ ]:
with open('../features/cnn_4_aug_1_fold8_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))


submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_4_aug_1_fold8.csv', index=False)

In [ ]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3), input_shape=(75, 75, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.3))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.3))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))
    return model
print('model model')


In [ ]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=120, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)

In [ ]:
with open('../features/cnn_5_aug_ang_fold3_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))


submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_5_aug_ang_fold3.csv', index=False)